In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

In [3]:
!pwd

/content/drive/MyDrive/Colab Notebooks


### 필요한 라이브러리 설치

In [4]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy validate

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-05 02:44:36.629104: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.4.4) =================
ℹ spaCy installation: /usr/local/lib/python3.8/dist-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_lg   >=3.4.0,<3.5.0   3.4.1   ✔
en_core_web_sm   >=3.4.0,<3.5.0   3.4.1   ✔



In [5]:
!python3 -m spacy download ko_core_news_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-06 03:47:35.965662: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 11.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_md')


### 라이브러리 임포트 및 데이터 불러오기

In [12]:
import pandas as pd
import math
import json
import spacy
from spacy.lang.ko.examples import sentences 

In [13]:
csv_sms = pd.read_csv('/content/drive/MyDrive/sms_backup/sms_miso.csv')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,8,41,54,55,67,77,92,95,98,99,104,105,109,110,113,114,115,118,120,121) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [15]:
df_sms = csv_sms[['body', 'contact_name', 'text']]

### 필터링1 : body가 NaN이 아닐 때 그 내용 & text가 NaN이 아닐 때 그 내용 => 모아서 데이터프레임 만들기

In [45]:
# 필터링1 : body가 NaN이 아닐 때 그 내용 & text가 NaN이 아닐 때 그 내용 => 모아서 데이터프레임 만들기
df_content = pd.DataFrame(columns=['content'])

for i in range(df_sms.shape[0]):
    if not pd.isna(df_sms['body'][i]):
        df_content = df_content.append(pd.DataFrame({'content' : [df_sms['body'][i]]}), ignore_index = True)  # 8481 개
    elif not pd.isna(df_sms['text'][i]):
        if '<smil>' not in df_sms['text'][i]:
            df_content = df_content.append(pd.DataFrame({'content' : [df_sms['text'][i]]}), ignore_index = True)  # 11214 개

In [17]:
df_content_copy = df_content.copy()

### Customizing Presidio Process

1. Deny-list based PII recognition  
2. Regex based PII recognition(O)    
3. Calling an external service for PII detection  


In [38]:
from typing import List
import re
from random import randint
import random

In [36]:
# ### Deny-list based PII recognition
# from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
# from presidio_analyzer.recognizer_registry import RecognizerRegistry
# from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts
# from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

# text1 = "Hi Miso, my phone number is 000-0000-0000."
# titles_list = ["Miso", "number", "name"]
# titles_recognizer = PatternRecognizer(supported_entity="TITLE", deny_list=titles_list)
# analyzer = AnalyzerEngine()
# analyzer.registry.add_recognizer(titles_recognizer)
# results = analyzer.analyze(text=text1, language="en")
# results

[type: TITLE, start: 3, end: 7, score: 1.0,
 type: TITLE, start: 18, end: 24, score: 1.0]

In [43]:
# print("Identified these PII entities:")
# for result in results:
#     print(f"- {text1[result.start:result.end]} as {result.entity_type}")

Identified these PII entities:
- Miso as TITLE
- number as TITLE


In [ ]:
### 1. Deny-list based PII recognition
# 개인정보 키워드 입력하여 해당 문자 제외
# 키워드 예시 : 본인이름, 전화번호, 지인이름, 은행이름, 통장번호, 생년월일, 집주소, 예전 집주소 등
df_content_2 = df_content[df_content['content'].str.contains('키워드1|키워드2|키워드3')==False] # 키워드 부분 변경
df_content_2 = df_content_2.reset_index(drop=True)

In [ ]:
### 2. Regex based PII recognition
# regex로 숫자(\d) 찾아내고 해당 개수만큼 랜덤 숫자로 대체
regex = r"\d+"
for idx in range(df_content_2.shape[0]):
    result = re.findall(regex, df_content_2['content'][idx])
    for r in result:
        a = str(randint(0,9))
        for i in range(len(r)-1):
            a += str(randint(0,9))
        df_content_2['content'][idx] = df_content_2['content'][idx].replace(r,a)
df_content_2

In [37]:
### 3. Calling spaCy for PII detection
nlp = spacy.load("ko_core_news_md")
sentences = "홍길동님 택배가 3-4시에 도착합니다"
doc = nlp(sentences)
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)
# PROPN : 고유명사, SCONJ : 종속접속사

홍길동님 택배가 3-4시에 도착합니다
홍길동님 PROPN compound
택배가 NOUN nsubj
3 NUM nummod
- X compound
4시에 ADV obl
도착합니다 SCONJ ROOT


In [41]:
df_content_3 = df_content_2.copy()

In [ ]:
### 3. Calling spaCy for PII detection
rep_list = ['홍길동', '나무꾼', '가나다']
nlp = spacy.load("ko_core_news_md")
for idx in range(df_content_3.shape[0]):
    doc = nlp(df_content_3['content'][idx])
    for token in doc:
        if token.pos_ == 'PROPN':
            df_content_3['content'][idx] = df_content_3['content'][idx].replace(token.text, random.choice(rep_list))
df_content_3

In [44]:
# 데이터프레임 개인 Gdrive 경로에 csv 파일로 저장
df_content_3.to_csv("/content/drive/MyDrive/sms_backup/sms_preprocessed_updated_miso.csv", header = False, index = False, encoding='utf-8-sig')